In [1]:
import os
os.getcwd()

'E:\\MLS\\grbg'

In [2]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [2]:
import pandas as pd
import numpy as np
import gradio
import librosa
import matplotlib.pyplot as plt
import librosa.display

In [3]:
from datasets import load_dataset
import datasets
ds = load_dataset("csv",data_files="./Datasets/musiccaps-public.csv", split='train',keep_in_memory=True)
ds

E:\MLS\grbg\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using custom data configuration default-eb916991cfa052bc
Found cached dataset csv (C:/Users/galih/.cache/huggingface/datasets/csv/default-eb916991cfa052bc/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


Dataset({
    features: ['ytid', 'start_s', 'end_s', 'audioset_positive_labels', 'aspect_list', 'caption', 'author_id', 'is_balanced_subset', 'is_audioset_eval'],
    num_rows: 5521
})

In [4]:
# JUST HELPER METHODS IN THIS CELL 
import subprocess
from pathlib import Path
def download_clip(
        video_identifier,
        output_filename,
        start_time,
        end_time,
        tmp_dir='/tmp/musiccaps',
        num_attempts=5,
        url_base='https://www.youtube.com/watch?v='):
        status = False

        command = f"""
            yt-dlp --quiet --no-warnings -x --audio-format wav -f bestaudio -o "{output_filename}" --download-sections "*{start_time}-{end_time}" {url_base}{video_identifier} --ffmpeg-location D:\\ffmpeg-master-latest-win64-gpl\\bin
        """.strip()

        attempts = 0
        while True:
            try:
                output = subprocess.check_output(command, shell=True,
                                                    stderr=subprocess.STDOUT)
            except subprocess.CalledProcessError as err:
                attempts += 1
                if attempts == num_attempts:
                    return status, err.output
            else:
                break

        # Check if the video was successfully saved.
        status = os.path.exists(output_filename)
        return status, 'Downloaded'
    

def process_data(example):
    outfile_path = str(data_dir / f"{example['ytid']}.wav")
    status = True
    if not os.path.exists(outfile_path):
        status = False
        status, log = download_clip(
            example['ytid'],
            outfile_path,
            example['start_s'],
            example['end_s'],
        )
        print(f"{outfile_path} - {log}")

    example['audio'] = outfile_path
    example['download_status'] = status
    return example

In [5]:

from datasets import Audio
cores = 4                 # How many processes to use for the loading
sampling_rate = 44100     # Sampling rate for the audio, keep in 44100
writer_batch_size = 1000  # How many examples to keep in memory per worker. Reduce if OOM.
data_dir = "./Datasets/gmusiccaps" # Where to save the data

# Just select some samples 
ds = ds.select(range(len(ds)))
# Create directory where data will be saved
data_dir = Path(data_dir)
data_dir.mkdir(exist_ok=True, parents=True)


ds = ds.map(
        process_data,
        num_proc=1,
        writer_batch_size=writer_batch_size,
        keep_in_memory=False
    ).filter(lambda x : x['download_status'] == True
    ).cast_column('audio', Audio(sampling_rate=sampling_rate))


  3%|██                                                                             | 143/5521 [00:06<04:23, 20.43ex/s]

Datasets\gmusiccaps\0J_2K1Gvruk.wav - b"ERROR: [youtube] 0J_2K1Gvruk: Private video. Sign in if you've been granted access to this video\n"


  6%|████▉                                                                          | 345/5521 [00:11<02:15, 38.08ex/s]

Datasets\gmusiccaps\0s0Uy0-zBa0.wav - Downloaded


 13%|██████████▌                                                                    | 742/5521 [00:16<01:51, 42.79ex/s]

...

 94%|█████████████████████████████████████████████████████████████████████████▎    | 5186/5521 [03:00<00:16, 20.66ex/s]

Datasets\gmusiccaps\vVNWjq9byoQ.wav - b"ERROR: [youtube] vVNWjq9byoQ: Private video. Sign in if you've been granted access to this video\n"


100%|██████████████████████████████████████████████████████████████████████████████| 5521/5521 [03:09<00:00, 29.18ex/s]


Datasets\gmusiccaps\xxCnmao8FAs.wav - b'\n\nERROR: ffmpeg exited with code 1\n'


100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 28.35ba/s]


In [6]:
ds[0]

{'ytid': '-0Gj8-vB1q4',
 'start_s': 30,
 'end_s': 40,
 'audioset_positive_labels': '/m/0140xf,/m/02cjck,/m/04rlf',
 'aspect_list': "['low quality', 'sustained strings melody', 'soft female vocal', 'mellow piano melody', 'sad', 'soulful', 'ballad']",
 'caption': 'The low quality recording features a ballad song that contains sustained strings, mellow piano melody and soft female vocal singing over it. It sounds sad and soulful, like something you would hear at Sunday services.',
 'author_id': 4,
 'is_balanced_subset': False,
 'is_audioset_eval': True,
 'audio': {'path': 'Datasets\\gmusiccaps\\-0Gj8-vB1q4.wav',
  'array': array([-0.00195292,  0.00100993,  0.00316163, ..., -0.01966176,
         -0.02357896,  0.        ], dtype=float32),
  'sampling_rate': 44100},
 'download_status': True}

In [7]:
import librosa
import IPython.display as ipd

ipd.Audio(ds[0]['audio']['array'],rate=ds[0]['audio']['sampling_rate'])

In [8]:
len(ds)

5491